In [ ]:
# using a conda environment Python=3.9
# running on Intel MacOS using Apple Metal
# using TensorFlow as ML framework
# versions:
from tensorflow import __version__ as tf_version
print("Python: ", sys.version)
print("TensorFlow: ", tf_version)

In [ ]:
# load MNIST using TF Load
import tensorflow_datasets as tfds
(train_ds, test_ds), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
# normalize the data
import tensorflow as tf
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255., label
train_ds = train_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
# build pipeline
BATCH_SIZE = 128
train_ds = train_ds.cache().shuffle(ds_info.splits['train'].num_examples).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# build model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])
model.summary()

In [ ]:
# compile model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
# train model
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)

In [ ]:
from tensorflow.keras.models import load_model
# save model
model.save('leads_model.h5')

# load model
model = load_model('leads_model.h5')

In [ ]:
# plot accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.show()